Лабораторна робота №4

Студента групи МІТ-31 (підгрупа 2)

Фуркала Олександра Олександровича

Завдання

In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv("insurance.csv")
print(data.describe())
print(data.isnull().sum())
# isnull використовується для створення нового DataFrame тієї самої форми, що й вихідний, але з Boolean значеннями. 
#У новому DataFrame кожен елемент має значення True, якщо відповідний елемент у вихідному DataFrame має значення null 
#(відсутній), і False в іншому випадку.
# sum() - у разі застосування до DataFrame, що містить Boolean значення, розглядає True як 1 і False як 0. 
print(data.head())

df = pd.get_dummies(data, columns=["sex", "smoker", "region"], prefix=["sex", "smoker", "region"])
# get_dummies створює нові двійкові (0 або 1) стовпці для кожної категорії в межах указаних категорійних стовпців.
#Кожен двійковий стовпець представляє наявність або відсутність певної категорії для певної точки даних.

scaler = StandardScaler()
numerical_cols = ["age", "bmi", "children"] 
df[numerical_cols] = scaler.fit_transform(df[numerical_cols].values.astype('float64'))
# scaler.fit_transform - цей метод застосовує стандартизаційне перетворення до вибраних даних. 
#Обчислює середнє значення та стандартне відхилення даних і відповідно масштабує їх.

columns_to_convert = ['sex_female', 'sex_male', 'smoker_no', 'smoker_yes', 'region_northeast', 'region_northwest', 'region_southeast', 'region_southwest']
for column in columns_to_convert:
    df[column] = df[column].astype('float64')

print(df.head())
df.to_csv('Lab5InsuranceAfter.csv')

for column in df.columns:
    data_type = df[column].dtype
    print(f"Data type of '{column}': {data_type}")

X = df.drop("charges", axis=1) 
y = df["charges"]  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=1)  
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1), loss='mean_absolute_error')
# learning_rate - невелике додатне число (зазвичай у діапазоні 0,1, 0,01, 0,001 тощо), 
#яке контролює розмір кроків, зроблених для оновлення параметрів моделі під час алгоритмів оптимізації. 

history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test))
# Збільшення кількості епох трохи підвищує ефективність моделі

mae = model.evaluate(X_test, y_test)
print(f"MAE on test data: {mae}")

               age          bmi     children       charges
count  1338.000000  1338.000000  1338.000000   1338.000000
mean     39.207025    30.663397     1.094918  13270.422265
std      14.049960     6.098187     1.205493  12110.011237
min      18.000000    15.960000     0.000000   1121.873900
25%      27.000000    26.296250     0.000000   4740.287150
50%      39.000000    30.400000     1.000000   9382.033000
75%      51.000000    34.693750     2.000000  16639.912515
max      64.000000    53.130000     5.000000  63770.428010
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64
   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     n